# Import packages   

In [4]:
import torch
import voice_to_text as vtt
import Mel_creator as mc

from data_utils_2 import TextMelLoader, TextMelCollate
from torch.utils.data.distributed import DistributedSampler
from logger import Tacotron2Logger
from distributed import apply_gradient_allreduce
from loss_function import Tacotron2Loss
from torch.cuda.amp import autocast, GradScaler

from torch.utils.data import DataLoader
import torch.distributed as dist
import time
from unidecode import unidecode
import math
from numpy import finfo
import os
import argparse
from model_FV import Tacotron2
import tqdm
from shutil import copytree
import matplotlib.pyplot as plt
import random as random

# Functions

In [25]:
def prepare_dataloaders(hparams):
    # Get data, data loaders and collate function ready
    trainset = TextMelLoader(hparams['training_files'], hparams['text_cleaners'],
                         hparams['max_wav_value'], hparams['sampling_rate'],
                         hparams['load_mel_from_disk'], hparams['filter_length'],
                         hparams['hop_length'], hparams['win_length'],
                         hparams['n_mel_channels'], hparams['mel_fmin'],
                         hparams['mel_fmax'], hparams['seed'])
    valset = TextMelLoader( hparams['validation_files'], hparams['text_cleaners'],
                         hparams['max_wav_value'], hparams['sampling_rate'],
                         hparams['load_mel_from_disk'], hparams['filter_length'],
                         hparams['hop_length'], hparams['win_length'],
                         hparams['n_mel_channels'], hparams['mel_fmin'],
                         hparams['mel_fmax'], hparams['seed'])
    collate_fn = TextMelCollate(n_frames_per_step=1)

    if hparams['distributed_run']:
        train_sampler = DistributedSampler(trainset)
        shuffle = False
    else:
        train_sampler = None
        shuffle = True

    train_loader = DataLoader(trainset, num_workers=1, shuffle=shuffle,
                              sampler=train_sampler,
                              batch_size=hparams['batch_size'], pin_memory=False,
                              drop_last=True, collate_fn=collate_fn)
    return train_loader, valset, collate_fn


def reduce_tensor(tensor, n_gpus):
    rt = tensor.clone()
    dist.all_reduce(rt, op=dist.reduce_op.SUM)
    rt /= n_gpus
    return rt

def init_distributed(n_gpus, rank, group_name):
    assert torch.cuda.is_available(), "Distributed mode requires CUDA."
    print("Initializing Distributed")

    # Set cuda device so everything is done on the right GPU.
    torch.cuda.set_device(rank % torch.cuda.device_count())

    # Initialize distributed communication
    dist.init_process_group(
        backend="gloo", init_method="tcp://localhost:54321",
        world_size=n_gpus, rank=rank, group_name=group_name)
    
    print("Done initializing distributed")

def prepare_directories_and_logger(output_directory, log_directory, rank):
    if rank == 0:
        if not os.path.isdir(output_directory):
            os.makedirs(output_directory)
            os.chmod(output_directory, 0o775)
        logger = Tacotron2Logger(os.path.join(output_directory, log_directory))
    else:
        logger = None
    return logger

def save_data(data, filename):
    with open(filename, 'w') as file:
        for line in data:
            file.write(' | '.join(line) + '\n')

def plot_alignment(alignment, info=None):
    fig, ax = plt.subplots(figsize=(10,8))
    im = ax.imshow(alignment, cmap='inferno', aspect='auto', origin='lower',
                   interpolation='none')
    ax.autoscale(enable=True, axis="y", tight=True)
    fig.colorbar(im, ax=ax)
    xlabel = 'Decoder timestep'
    if info is not None:
        xlabel += '\n\n' + info
    plt.xlabel(xlabel)
    plt.ylabel('Encoder timestep')
    plt.tight_layout()
    fig.canvas.draw()
    plt.show()


##Funciones en construccion
#########################



def load_model(hparams):
    model = Tacotron2().cuda()  ##Josue
    if hparams['fp16_run']:
        model.decoder.attention_layer.score_mask_value = ('float16').min

    if hparams['distributed_run']:
        model = apply_gradient_allreduce(model)

    return model


def warm_start_model(checkpoint_path, model, ignore_layers):
    assert os.path.isfile(checkpoint_path)
    print("Warm starting model from checkpoint '{}'".format(checkpoint_path))
    checkpoint_dict = torch.load(checkpoint_path, map_location='cpu')
    model_dict = checkpoint_dict['state_dict']
    if len(ignore_layers) > 0:
        model_dict = {k: v for k, v in model_dict.items()
                      if k not in ignore_layers}
        dummy_dict = model.state_dict()
        dummy_dict.update(model_dict)
        model_dict = dummy_dict
    model.load_state_dict(model_dict)
    return model


def load_checkpoint(checkpoint_path, model, optimizer):
    assert os.path.isfile(checkpoint_path)
    print("Loading checkpoint '{}'".format(checkpoint_path))
    checkpoint_dict = torch.load(checkpoint_path, map_location='cpu')
    model.load_state_dict(checkpoint_dict['state_dict'])
    optimizer.load_state_dict(checkpoint_dict['optimizer'])
    learning_rate = checkpoint_dict['learning_rate']
    iteration = checkpoint_dict['iteration']
    print("Loaded checkpoint '{}' from iteration {}" .format(
        checkpoint_path, iteration))
    return model, optimizer, learning_rate, iteration


def save_checkpoint(model, optimizer, learning_rate, iteration, filepath):
    print("Saving model and optimizer state at iteration {} to {}".format(
        iteration, filepath))
    try:
        torch.save({'iteration': iteration,
                'state_dict': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'learning_rate': learning_rate}, filepath)
    except KeyboardInterrupt:
        print("interrupt received while saving, waiting for save to complete.")
        torch.save({'iteration': iteration,'state_dict': model.state_dict(),'optimizer': optimizer.state_dict(),'learning_rate': learning_rate}, filepath)
    print("Model Saved")






########################
##Validation process

def validate(model, criterion, valset, iteration, batch_size, n_gpus,
             collate_fn, logger, distributed_run, rank, epoch, start_eposh, learning_rate,sample_interbal, save_audio = False, audio_path = None):
    """Handles all the validation scoring and printing"""
    model.eval()
    with torch.no_grad():
        val_sampler = DistributedSampler(valset) if distributed_run else None
        val_loader = DataLoader(valset, sampler=val_sampler, num_workers=1,
                                shuffle=False, batch_size=batch_size,
                                pin_memory=False, collate_fn=collate_fn)

        val_loss = 0.0
        for i, batch in enumerate(val_loader):
            x, y = model.parse_batch(batch)
            y_pred = model(x)
            loss = criterion(y_pred, y)
            if distributed_run:
                reduced_val_loss = reduce_tensor(loss.data, n_gpus).item()
            else:
                reduced_val_loss = loss.item()
            val_loss += reduced_val_loss
        val_loss = val_loss / (i + 1)

    model.train()
    if rank == 0:
        print("Epoch: {} Validation loss {}: {:9f}  Time: {:.1f}m LR: {:.6f}".format(epoch, iteration, val_loss,(time.perf_counter()-start_eposh)/60, learning_rate))
        logger.log_validation(val_loss, model, y, y_pred, iteration)
        _, mel_outputs, gate_outputs, alignments = y_pred
        idx = random.randint(0, alignments.size(0) - 1)
        plot_alignment(alignments[idx].data.cpu().numpy().T)




        
##Training process

def train( log_directory, checkpoint_path, warm_start, n_gpus,
          rank, group_name, hparams, log_directory2):
    """Training and validation logging results to tensorboard and stdout

    Params
    ------
    log_directory (string) directory to save tensorboard logs
    checkpoint_path(string): checkpoint path
    n_gpus (int): number of gpus
    rank (int): rank of current gpu
    hparams (object): comma separated list of "name=value" pairs.
    """
    if hparams['distributed_run']:
        init_distributed(n_gpus, rank, group_name)

    torch.manual_seed(hparams['seed'])
    torch.cuda.manual_seed(hparams['seed'])

    model = load_model(hparams)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=hparams['learning_rate'],
                                 weight_decay=hparams['weight_decay'])

    if hparams['fp16_run']:
        scaler = GradScaler()
    if hparams['distributed_run']:
        model = apply_gradient_allreduce(model)

    criterion = Tacotron2Loss()

    logger = prepare_directories_and_logger(
        hparams['ouputh_checkpoint_path'], hparams['log_directory_1'], hparams['rank'])

    train_loader, valset, collate_fn = prepare_dataloaders(hparams)

    # Load checkpoint if one exists
    iteration = 0
    epoch_offset = 0
    if checkpoint_path is not None and os.path.isfile(checkpoint_path):
        if warm_start:
            model = warm_start_model(
                checkpoint_path, model, hparams['ignore_layers'])
        else:
            model, optimizer, _learning_rate, iteration = load_checkpoint(
                checkpoint_path, model, optimizer)
            if hparams['use_saved_learning_rate']:
                learning_rate = _learning_rate
            iteration += 1  # next iteration is iteration + 1
            epoch_offset = max(0, int(iteration / len(train_loader)))
    else:
      model = warm_start_model(hparams['ouputh_checkpoint_path'], model, hparams['ignore_layers'])
      # download LJSpeech pretrained model if no checkpoint already exists
    
    start_eposh = time.perf_counter()
    learning_rate = 0.0
    model.train()
    is_overflow = False
    # ================ MAIN TRAINNIG LOOP! ===================
    for epoch in tqdm(range(epoch_offset, hparams['epochs'])):
        print("\nStarting Epoch: {} Iteration: {}".format(epoch, iteration))
        start_eposh = time.perf_counter() # eposh is russian, not a typo
        for i, batch in tqdm(enumerate(train_loader), total=len(train_loader)):
            start = time.perf_counter()
            if iteration < hparams['decay_start']: learning_rate = 5e-4
            else: iteration_adjusted = iteration - hparams['decay_start']; learning_rate = (5e-4*(e**(-iteration_adjusted/8000))) + 0
            learning_rate = max(hparams['min_learning_rate'] , learning_rate) # output the largest number
            for param_group in optimizer.param_groups:
                param_group['lr'] = learning_rate

            model.zero_grad()
            x, y = model.parse_batch(batch)
            y_pred = model(x)
            with autocast(enabled= hparams['fp16_run']):
                y_pred = model(x)

            loss = criterion(y_pred, y)

            if hparams['distributed_run']:
                reduced_loss = reduce_tensor(loss.data, n_gpus).item()
            else:
                reduced_loss = loss.item()
            if  hparams['fp16_run']:
                scaler.scale(loss).backward()
            else:
                loss.backward()

            if  hparams['fp16_run']:
                grad_norm = torch.nn.utils.clip_grad_norm_(
                    model.parameters(), max_norm=1.0)
                is_overflow = math.isnan(grad_norm)
                if not is_overflow:
                        scaler.step(optimizer)
                        scaler.update()
            else:
                grad_norm = torch.nn.utils.clip_grad_norm_(
                    model.parameters(), max_norm=1)
                optimizer.step()

            if not is_overflow and rank == 0:
                duration = time.perf_counter() - start
                logger.log_training(
                    reduced_loss, grad_norm, learning_rate, duration, iteration)
                print("Batch {} loss {:.6f} Grad Norm {:.6f} Time {:.6f}".format(iteration, reduced_loss, grad_norm, duration), end='\r', flush=True)

            iteration += 1
        validate(model, criterion, valset, iteration,
                 hparams['batch_size'], n_gpus, collate_fn, logger,
                 hparams['distributed_run'], rank, epoch, start_eposh, learning_rate)
        save_checkpoint(model, optimizer, learning_rate, iteration, checkpoint_path)
        if log_directory2 is not None:
            copytree(log_directory, log_directory2)



# Training process   

### Hparams

In [26]:
hparams = {
    'in_audio_path':"E:\\wavs",
    'out_audio_path':"E:\\wavs_text",
    'whisper_model':'base',
    'whipser_language':'english',
    'out_audio_mels_path':"E:\\mels",
    'ouputh_checkpoint_path':'E:\\CH_output\\pretrained_model.pt',
    'log_directory_1':"E:\\loggs_1",
    'log_directory_2':'E:\\loggs_2',
    ################################
    # Data Parameters              #
    ################################
    'training_files':'E:\\wavs_text\\train.txt',
    'validation_files':'E:\\wavs_text\\val.txt',
    'n_gpus':1,
    'rank':1,
    'group_name':None,
    'text_cleaners': ['english_cleaners'],
    ################################
    # Audio Parameters             #
    ################################
    'max_wav_value': 32768.0,
    'sampling_rate': 22050,
    'load_mel_from_disk': False,
    'filter_length': 1024,
    'hop_length': 256,
    'win_length': 1024,
    'n_mel_channels': 80,
    'mel_fmin': 0.0,
    'mel_fmax': 8000.0,
    'seed': 20,
     ################################
     # Optimization Hyperparameters #
    ################################
    'use_saved_learning_rate':False,
    'learning_rate':1e-3,
    'weight_decay':1e-6,
    'grad_clip_thresh':1.0,
    'batch_size':64,
    'mask_padding':True,# set model's padded outputs to padded values
    ################################
    # Experiment Parameters        #
    ################################
    'epochs':500,
    'iters_per_checkpoint':1000,
    'dynamic_loss_scaling':True,
    'fp16_run':False,
    'distributed_run':False,
    'dist_backend':"gloo",
    'dist_url':"tcp://localhost:54321",
    'cudnn_enabled':True,
    'cudnn_benchmark':True,
    'ignore_layers':['embedding.weight'],
    'decay_start': 15000,
    'min_learning_rate': 1e-5
}


#### Step 1 create Text from audio  

In [4]:
vtt.voice_to_text(hparams['in_audio_path'],hparams['out_audio_path'],language="english") #Here creates the audio files to a text file

100%|██████████| 13100/13100 [1:12:05<00:00,  3.03it/s]


#### Step 2 Audio To mels   

In [6]:
mc.mel_creator(hparams['in_audio_path'],hparams['out_audio_mels_path']) #Here creates the audio files to Mel tensor's

Mels state: 100%|██████████| 13100/13100 [3:26:20<00:00,  1.06it/s]  

wavs_mel created


In [15]:
checkpoint_path = hparams['ouputh_checkpoint_path']

In [124]:
#Crete training and validation datasets
data = []
with open('E:\\wavs_text\\wavs_text.txt', 'r') as file:
    for line in file:
        data.append(line.strip().split(' | '))
import random

random.shuffle(data)  # Mezclar los datos aleatoriamente

# Calcular los tamaños de los conjuntos de datos
total_samples = len(data)
train_size = int(0.7 * total_samples)
val_size = int(0.30 * total_samples)

# Dividir los datos en conjuntos de entrenamiento, validación y prueba
train_data = data[:train_size]
val_data = data[train_size:train_size + val_size]



save_data(train_data, 'E:\\wavs_text\\train.txt')
save_data(val_data, 'E:\\wavs_text\\val.txt')





In [27]:
train(log_directory=hparams['log_directory_1'], checkpoint_path=checkpoint_path,
       warm_start=True,n_gpus=hparams['n_gpus'], rank=hparams['rank'],group_name= hparams['group_name'], 
       hparams=hparams, log_directory2=hparams['log_directory_2'])

Warm starting model from checkpoint 'E:\CH_output\pretrained_model.pt'


RuntimeError: Error(s) in loading state_dict for Tacotron2:
	Unexpected key(s) in state_dict: "encoder.convolutions.1.0.conv.weight", "encoder.convolutions.1.0.conv.bias", "encoder.convolutions.1.1.weight", "encoder.convolutions.1.1.bias", "encoder.convolutions.1.1.running_mean", "encoder.convolutions.1.1.running_var", "encoder.convolutions.1.1.num_batches_tracked", "encoder.convolutions.2.0.conv.weight", "encoder.convolutions.2.0.conv.bias", "encoder.convolutions.2.1.weight", "encoder.convolutions.2.1.bias", "encoder.convolutions.2.1.running_mean", "encoder.convolutions.2.1.running_var", "encoder.convolutions.2.1.num_batches_tracked". 

In [ ]:
def load_model(hparams):
    model = Tacotron2().cuda()  ##Josue
    if hparams['fp16_run']:
        model.decoder.attention_layer.score_mask_value = ('float16').min

    if hparams['distributed_run']:
        model = apply_gradient_allreduce(model)

    return model

In [28]:
warm_start_model(hparams['ouputh_checkpoint_path'], load_model(hparams), hparams['ignore_layers'])

Warm starting model from checkpoint 'E:\CH_output\pretrained_model.pt'


RuntimeError: Error(s) in loading state_dict for Tacotron2:
	Unexpected key(s) in state_dict: "encoder.convolutions.1.0.conv.weight", "encoder.convolutions.1.0.conv.bias", "encoder.convolutions.1.1.weight", "encoder.convolutions.1.1.bias", "encoder.convolutions.1.1.running_mean", "encoder.convolutions.1.1.running_var", "encoder.convolutions.1.1.num_batches_tracked", "encoder.convolutions.2.0.conv.weight", "encoder.convolutions.2.0.conv.bias", "encoder.convolutions.2.1.weight", "encoder.convolutions.2.1.bias", "encoder.convolutions.2.1.running_mean", "encoder.convolutions.2.1.running_var", "encoder.convolutions.2.1.num_batches_tracked". 

In [34]:
model = load_model(hparams)

In [35]:
model

Tacotron2(
  (embedding): Embedding(256, 512)
  (encoder): Encoder(
    (convolutions): ModuleList(
      (0): Sequential(
        (0): ConvNorm(
          (conv): Conv1d(512, 512, kernel_size=(5,), stride=(1,), padding=(2,))
        )
        (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (lstm): LSTM(512, 256, batch_first=True, bidirectional=True)
  )
  (decoder): Decoder(
    (prenet): Prenet(
      (layers): ModuleList(
        (0): LinearNorm(
          (linear_layer): Linear(in_features=80, out_features=256, bias=False)
        )
        (1): LinearNorm(
          (linear_layer): Linear(in_features=256, out_features=256, bias=False)
        )
      )
    )
    (attention_rnn): LSTMCell(768, 1024)
    (attention_layer): Attention(
      (query_layer): LinearNorm(
        (linear_layer): Linear(in_features=1024, out_features=128, bias=False)
      )
      (memory_layer): LinearNorm(
        (linear_layer): Linear(in_features=

In [36]:
model_dict = torch.load(hparams['ouputh_checkpoint_path'], map_location='cpu')
model.load_state_dict(model_dict)

# Establece el modo de evaluación
model.eval()


RuntimeError: Error(s) in loading state_dict for Tacotron2:
	Missing key(s) in state_dict: "embedding.weight", "encoder.convolutions.0.0.conv.weight", "encoder.convolutions.0.0.conv.bias", "encoder.convolutions.0.1.weight", "encoder.convolutions.0.1.bias", "encoder.convolutions.0.1.running_mean", "encoder.convolutions.0.1.running_var", "encoder.lstm.weight_ih_l0", "encoder.lstm.weight_hh_l0", "encoder.lstm.bias_ih_l0", "encoder.lstm.bias_hh_l0", "encoder.lstm.weight_ih_l0_reverse", "encoder.lstm.weight_hh_l0_reverse", "encoder.lstm.bias_ih_l0_reverse", "encoder.lstm.bias_hh_l0_reverse", "decoder.prenet.layers.0.linear_layer.weight", "decoder.prenet.layers.1.linear_layer.weight", "decoder.attention_rnn.weight_ih", "decoder.attention_rnn.weight_hh", "decoder.attention_rnn.bias_ih", "decoder.attention_rnn.bias_hh", "decoder.attention_layer.query_layer.linear_layer.weight", "decoder.attention_layer.memory_layer.linear_layer.weight", "decoder.attention_layer.v.linear_layer.weight", "decoder

# WAVEGLOW

In [14]:
MAX_WAV_VALUE = 32768.0
mel_files = os.listdir("E:\\Github\\F-voice\\F-VOICE\\train_model_0\\waveglow\\mel_spectograms")
mel_files

['LJ001-0001.npy']

In [15]:
import os
from scipy.io.wavfile import write
import torch
from torch.cuda.amp import autocast, GradScaler

from denoiser import Denoiser

In [73]:
import numpy as np
def main(mels_files, waveglow_path, sigma, output_dir, sampling_rate, is_fp16, denoiser_strength):
    mels = os.listdir(mel_files)
    waveglow = torch.load(waveglow_path)['model']
    waveglow = waveglow.remove_weightnorm(waveglow)
    waveglow = waveglow.cuda().eval()

    if denoiser_strength > 0:
        denoiser = Denoiser(waveglow).cuda()

    scaler = torch.cuda.amp.GradScaler()  # Create a GradScaler for mixed precision

    for i, file_path in enumerate(os.listdir(mels_files)):
        file_name = os.listdir(mel_files)[0]
        mel = np.load(mel_files+"\\"+file_name)  # Load the .npy file as a NumPy array
        mel = torch.FloatTensor(mel).cuda()  # Convert the NumPy array to a PyTorch tensor and move it to GPU
        mel = torch.unsqueeze(mel, 0)
        mel = mel.half() if is_fp16 else mel
        print(mel.shape)

        with torch.no_grad():
            # Use torch.cuda.amp.autocast for mixed precision
            if is_fp16:
                with torch.cuda.amp.autocast():
                    audio = waveglow.infer(mel, sigma=sigma)
                    if denoiser_strength > 0:
                        audio = denoiser(audio, denoiser_strength)
                    audio = audio * MAX_WAV_VALUE
            else:
                audio = waveglow.infer(mel, sigma=sigma)
                if denoiser_strength > 0:
                    audio = denoiser(audio, denoiser_strength)
                audio = audio * MAX_WAV_VALUE

        audio = audio.squeeze()
        audio = audio.cpu().numpy()
        audio = audio.astype('int16')
        audio_path = os.path.join(output_dir, "{}_synthesis.wav".format(file_name))
        write(audio_path, sampling_rate, audio)
        print(audio_path)


In [74]:
main(mels_files="E:\\Github\\F-voice\\F-VOICE\\train_model_0\\waveglow\\mel_spectograms", waveglow_path="E:\\Github\\F-voice\\F-VOICE\\train_model_0\\waveglow_256channels.pt", sigma=0.6,output_dir="E:\\Github\\F-voice\\F-VOICE\\train_model_0\\waveglow\\mel_spectograms" ,
         sampling_rate=22050, is_fp16 = True, denoiser_strength = 0.0)

c:\Users\Laptop Bonita\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\serialization.py:1113: SourceChangeWarning: source code of class 'torch.nn.modules.conv.ConvTranspose1d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
c:\Users\Laptop Bonita\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\serialization.py:1113: SourceChangeWarning: source code of class 'torch.nn.modules.container.ModuleList' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
c:\Users\Laptop Bonita\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\serialization.py:1113: SourceChangeWarning: source code of class 't

torch.Size([1, 513, 832])


RuntimeError: Given transposed=1, weight of size [80, 80, 1024], expected input[1, 513, 832] to have 80 channels, but got 513 channels instead